# Interactive Session: Hazards 


We are now ready use our knowledge of integrating human and environmental geospatial datasets to understand exposure to natural hazards. We combine what we have learned so far to measure how many people were exposed to the devastating [2015 Nepal Earthquake](https://en.wikipedia.org/wiki/April_2015_Nepal_earthquake). This analysis comes from a paper ([link here](https://www.mdpi.com/1171374?trk=public_post_reshare-text)) published a few years ago. We look at how measures of exposure to three different natural disasters to illustrate how population exposure estimates can vary depending on the human dataset you use. This has big policy implications from resource allocation during response, to figuring out how much money is needed to rebuild.  

In this session, we will use [five gridded population datasets](https://www.popgrid.org) and [earthquake data from USGS](https://earthquake.usgs.gov/earthquakes/eventpage/us20002ejl/executive). Gridded population datasets combine census data with remote-sensed and GIS data to infer high resolution population estimates in locations where we have a poor understanding of the population distribution. Broadly, there are two methods to create gridded population datasets: top-down and bottom-up. Top-down approaches use allocate coarse-grained census data to high-resolution grid cells based on where satellite/GIS data tells us where we think people live. Bottom-up approaches link high-resolution micro-census data to satellite/GIS data and then use computer vision algorithms to create populations maps (for more info, including the source of the image below, see [WorldPop](https://www.worldpop.org/methods/populations/). But because gridded population producers use different in-put data and algorithms, they produce different population estimates. This can create real-world proplems, like trying to measure how many people are effected by a major earthquake in an area lacking recent public, high-resolution census data. 

In this tutorial, we will be using a new package caller [`rioxarray`](https://corteva.github.io/rioxarray/stable/) which builds upon `rasterio` and is a bit more user friendly.

<img src="./assets/wp.png" alt="rastervector" width="1500"/>
 
<p style="height:1pt"> </p>

<div class="boxhead2">
    Session Topics
</div>

<div class="boxtext2">
<ul class="a">
    <li> 📌 Introduction to <span class="codeb">Rasterio</span> </li>
    <ul class="b">
        <li> Resampling raster data </li>
        <li> Clipping raster data </li>
        <li> Reprojecting raster data </li>   
        <li> Rasters math </li>   
</ul>
</div>

<hr style="border-top: 0.2px solid gray; margin-top: 12pt; margin-bottom: 0pt"></hr>

### Instructions
We will work through this notebook together. To run a cell, click on the cell and press "Shift" + "Enter" or click the "Run" button in the toolbar at the top. 

<p style="color:#408000; font-weight: bold"> 🐍 &nbsp; &nbsp; This symbol designates an important note about Python structure, syntax, or another quirk.  </p>

<p style="color:#008C96; font-weight: bold"> ▶️ &nbsp; &nbsp; This symbol designates a cell with code to be run.  </p>

<p style="color:#008C96; font-weight: bold"> ✏️ &nbsp; &nbsp; This symbol designates a partially coded cell with an example.  </p>

<hr style="border-top: 1px solid gray; margin-top: 24px; margin-bottom: 1px"></hr>

In [ ]:
# dependencies 
import os 
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio 
import rioxarray as rio
from glob import glob
from rasterio.enums import Resampling
import xarray as xr
from scipy.stats import variation 
import rasterio.mask
from rasterstats import zonal_stats, gen_zonal_stats
import matplotlib.pyplot as plt
import matplotlib.patches as patches
Patch = patches.Patch

### Resampling raster data
First, if we want do identify rural and urban populations, we need a quasi indepedent dataset. Here we're going to an urban-rural binary land cover classification derived from MODIS data—the MODIS global urban extent product [(MGUP)](https://www.sciencedirect.com/science/article/pii/S0303243420308989). But the MGUP data is produced at 250m, where as our population data is 1-km spatial resolution.  

We need to resample the MGUP data using `rasterio`. We are going to use `mode` since this is a binaru `rural-urban` classified raster and we need to upsample it. Here is a function to do this.
<div class="run">
    ▶️ <b> Run the cells below. </b>
</div>

In [ ]:
def resample(fn_in, fn_out, scale_factor, method):
    
    """ Resamples a raster and save it out
    Args:
        fn_in = file path and name of tif input as str
        fn_out = file path and name of tif output as str 
        scale_factor = factor to up or down scale a pixel as float
        method = method to resample (rasterio object), see rasterio documentation
    """
    
    with rasterio.open(fn_in) as dataset:

        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=method
        )

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
    
    # meta data to write out
    out_meta = dataset.meta

    # Update meta data
    out_meta.update({"driver": "GTiff",
             "height": data.shape[1],
             "width": data.shape[2],
             "transform": transform})

    # write image 
    with rasterio.open(fn_out, "w", **out_meta) as dest:
        dest.write(data)

In [ ]:
# File paths
data_in = os.path.join('../data/tutorial011/MGUP_annual_2001_2018/')
data_out = os.path.join('./data/')

In [ ]:
# File names
modis_in = os.path.join(data_in+'MGUP_annual_2001_2018/MGUP_2015.tif')
modis_out = os.path.join(data_out+'MGUP_2015-1km.tif')

In [ ]:
# Resample and save the MODIS landcover raster
resample(modis_in, modis_out, 0.5, Resampling.mode)

### Clipping raster data

The next step is to clip population rasters to Nepal's boundaries. This will reduce our memory use a lot and speed things up - the population rasters are quite large. 

We will again use `rasterio` to clip our population rasters to a shape file for Nepal. Here is a function to do this.
<div class="run">
    ▶️ <b> Run the cells below. </b>
</div>

In [1]:
# Functions
def raster_clip(rst_fn, polys, fn_out):
    
    """ function clips a raster and saves it out
    args:
        rst_fn = raster you want to clip
        polys = polys you want to clip to
        fn_out = path/to/file/out ... name of file you want to save 
    """
    
    # clip raster
    with rasterio.open(rst_fn) as src:
        out_image, out_transform = rasterio.mask.mask(src, polys, crop=True)
        out_meta = src.meta
        
    # Update meta data
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    # write image 
    with rasterio.open(fn_out, "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
# Set up the file paths for each raster 
wp_path = os.path.join(data_in + 'WorldPop16/ppp_2016_1km_Aggregated.tif')
ls_path = os.path.join(data_in + 'LandScan-Global-2015/lspop2015/w001001.adf')
esri_path = os.path.join(data_in + 'ESRI_WPE_2016_Pop/WPE_1KM_2016_Pop.tif')
ghs_path = os.path.join(data_in + 'GHS-Pop/GHS_POP_E2015_GLOBE_R2019A_4326_30ss_V1_0.tif')
gpw_path = os.path.join(data_in + 'gpw_v4/gpw-v4-population-count-rev11_2015_30_sec_tif/gpw_v4_population_count_rev11_2015_30_sec.tif')
modis_path = os.path.join(data_out + 'MGUP_2015-1km.tif')

In [ ]:
# Open shape file
polys_fn = os.path.join(data_in + 'GPWv4-boundaries/gwpv4_npl_admin4.shp')
polys = gpd.read_file(polys_fn)
shapes = polys["geometry"]

In [ ]:
shapes.head()

<div class="example">
    ✏️ <b> Try it. </b> 
   Plot the Nepal shape file. 
</div>

In [ ]:
# your code here ... 

**Now clip the rasters**

In [ ]:
# Clip WorldPop Data
out = os.path.join(data_out, 'wp2016_npl.tif')
print(out)
raster_clip(rst_fn = ls_path, polys = shapes, fn_out = out)

In [ ]:
# Clip LandScan Data
out = os.path.join(data_out, 'ls2015_npl.tif')
print(out)
raster_clip(rst_fn = wp_path, polys = shapes, fn_out = out)

In [ ]:
# Clip ESRI Data
out = os.path.join(data_out, 'esri2015_npl.tif')
print(out)
raster_clip(rst_fn = esri_path, polys = shapes, fn_out = out)

In [ ]:
# Clip GHS Data
out = os.path.join(data_out, 'ghs2015_npl.tif')
print(out)
raster_clip(rst_fn = ghs_path, polys = shapes, fn_out = out)

In [ ]:
# Clip GPW Data
out = os.path.join(data_out, 'gpw2015_npl.tif')
print(out)
raster_clip(rst_fn = gpw_path, polys = shapes, fn_out = out)

In [ ]:
out = os.path.join(data_out, 'modis2015_npl.tif')
print(out)
raster_clip(rst_fn = modis_path, polys = shapes, fn_out = out)

In [ ]:
# your code here ... 

Now clip the rasters

In [ ]:
# Clip WorldPop Data
out = os.path.join(data_out, 'wp2016_npl.tif')
print(out)
raster_clip(rst_fn = ls_path, polys = shapes, fn_out = out)

In [ ]:
# Clip LandScan Data
out = os.path.join(data_out, 'ls2015_npl.tif')
print(out)
raster_clip(rst_fn = wp_path, polys = shapes, fn_out = out)

In [ ]:
# Clip ESRI Data
out = os.path.join(data_out, 'esri2015_npl.tif')
print(out)
raster_clip(rst_fn = esri_path, polys = shapes, fn_out = out)

In [ ]:
# Clip GHS Data
out = os.path.join(data_out, 'ghs2015_npl.tif')
print(out)
raster_clip(rst_fn = ghs_path, polys = shapes, fn_out = out)

In [ ]:
# Clip GPW Data
out = os.path.join(data_out, 'gpw2015_npl.tif')
print(out)
raster_clip(rst_fn = gpw_path, polys = shapes, fn_out = out)

In [ ]:
# Clip the MGUP raster
out = os.path.join(data_out, 'modis2015_npl.tif')
print(out)
raster_clip(rst_fn = modis_path, polys = shapes, fn_out = out)

### Spatially align (match) and reproject the population rasters

Now, we need to match and reproject the population rasters. In other words, we need to eeproject popgrid rasters into the same CRS, size and projection so they stack. But first, let's take a look at the `crs` and `shape` of each raster.


<div class="run">
    ▶️ <b> Run the cells below. </b>
</div>

In [ ]:
# MODIS2015_LCType2_1km-urban.tif is 2015 MODIS from GEE reclassified to urban/rural 
# modis : height: 31982: 80148, MODIS Sinusoidal
modis = rio.open_rasterio(os.path.join(data_out +'./modis2015_npl.tif'))
modis

In [ ]:
# GPW v4 : width = 43200 height = 21600 epsg = 4326
gpw = rio.open_rasterio(os.path.join(data_out + './gpw2015_npl.tif'))
gpw

In [ ]:
# World Pop 2016 (?_): width = 43200 height = 18720 epsg = 4326
wp = rio.open_rasterio(os.path.join(data_out + './wp2016_npl.tif'))
wp

In [ ]:
# ESRI 2016 : width = 40074 height = 14285 epsg = 4326
esri = rio.open_rasterio(os.path.join(data_out + './esri2015_npl.tif'))
esri

In [ ]:
# LS 2015 : width = 43200 height = 21600 epsg = 4326
ls = rio.open_rasterio(os.path.join(data_out + './ls2015_npl.tif'))
ls

In [ ]:
# GHS 2015 : width = 43200 height = 21600 epsg = 4326
ghs = rio.open_rasterio(os.path.join(data_out + './ghs2015_npl.tif')
ghs

The height and shape is ...? 
- MODIS
- GPW
- WP
- ESRI
- LS
- GHS

Now, let's check the nan values for all data...This is the meta data flag that says "this value means nan".

In [ ]:
print('modis nan value is', modis.data[0][0][0])
print('GPW nan value is', gpw.data[0][0][0])
print('World Pop nan value is', wp.data[0][0][0])
print('LandScan nan value is', ls.data[0][0][0])
print('ESRI nan value is', esri.data[0][0][0])
print('GHS nan value is', ghs.data[0][0][0])

**Check the nan fill values for all data** <br>
This is the meta data flag that says "this value means nan" _and_ print this value when we see an `nan'. 

In [ ]:
# modis doesn't have an assigned fill value so we make one
modis.attrs['_FillValue'] = 0

# Check fille values
print(modis.attrs['_FillValue'])
print(gpw.attrs['_FillValue'])
print(esri.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])
print(ghs.attrs['_FillValue'])

In [ ]:
# Set all NAs and -999s to Zero
modis.data = np.where(modis.data < 1,-999, modis.data)
gpw.data = np.where(gpw.data < 1,-999, gpw.data)
wp.data = np.where(wp.data < 1,-999, wp.data)
ls.data = np.where(ls.data < 1,-999, ls.data)
esri.data = np.where(esri.data < 1,-999, esri.data)
ghs.data = np.where(ghs.data < 1,-999, ghs.data)

In [ ]:
# Check the nan data
print('modis na value is', modis.data[0][0][0])
print('GPW na value is', gpw.data[0][0][0])
print('World Pop na value is', wp.data[0][0][0])
print('LandScan na value is', ls.data[0][0][0])
print('ESRI na value is', esri.data[0][0][0])
print('GHS na value is', ghs.data[0][0][0])

In [ ]:
# Check nan fill values in the meta data
print(modis.attrs['_FillValue'])
print(gpw.attrs['_FillValue'])
print(esri.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])
print(ghs.attrs['_FillValue'])

In [ ]:
# Set all nan fill values to - 999 in the meta data
na_val = -999

modis.attrs['_FillValue'] = na_val
gpw.attrs['_FillValue'] = na_val
esri.attrs['_FillValue'] = na_val
ls.attrs['_FillValue'] = na_val
wp.attrs['_FillValue'] = na_val
ghs.attrs['_FillValue'] = na_val

In [ ]:
# Check nan fill values in the meta data
print(modis.attrs['_FillValue'])
print(gpw.attrs['_FillValue'])
print(esri.attrs['_FillValue'])
print(ls.attrs['_FillValue'])
print(wp.attrs['_FillValue'])
print(ghs.attrs['_FillValue'])

**Reproject rasters** <br>
Now we pick one raster, and reproject and match the other rasters to a single basemap. This choice is arbitrary and could use sensitivity analysis, but we'll use GPWv4 because it is the underlying data for many of the other products.

In [ ]:
# Reproject all datasets to GPWv4
modis_match = modis.rio.reproject_match(gpw)
gwp_match = ghs.rio.reproject_match(gpw) # this is not needed per say, but we're doing it for consistancy
ls_match = ls.rio.reproject_match(gpw)
esri_match = esri.rio.reproject_match(gpw)
wp_match = wp.rio.reproject_match(gpw)
ghs_match = ghs.rio.reproject_match(gpw)

**Now check the reprojected data**

In [ ]:
modis_match 

In [ ]:
gwp_match

In [ ]:
ls_match 

In [ ]:
esri_match 

In [ ]:
wp_match 

In [ ]:
ghs_match 

**Now write the reprojected and matched rasters to disk**

Notice that `MGUP15_2015-1km-matched.tif` uses `-` not `_` ... this helps later when we want to use `glob` to get the files we want to work with.

In [ ]:
# Now write the reprojected and matched rasters
modis_match.rio.to_raster(data_out + 'MGUP15_2015-1km-matched.tif') 
gpw.rio.to_raster(data_out + 'GPWv4_matched.tif')
esri_match.rio.to_raster(data_out + 'ESRI16_matched.tif')
ls_match.rio.to_raster(data_out + 'LS15_matched.tif')
wp_match.rio.to_raster(data_out + 'WP16_matched.tif')
ghs_match.rio.to_raster(data_out + 'GHS15_matched.tif')

### Part 2
Make rural and urban masked rasters

In [ ]:
def raster_mask(rst_fn, lc_arr, data_out):
    """ Writes out masked rural and urban populations from pop rasters
    Args:
        rst_nm = pop raster file name + path
        lc_fn = urban/rural bianary w/ urban =1, rural = -999
        data_out = path to write out new rsts
    """
    
    # split for naming
    rst_nm = rst_fn.split('/')[1].split('.tif')[0]
    print(rst_nm)
    
    # open pop rasters and get array
    arr = rasterio.open(rst_fn).read(1)
    
    # multiply for urban
    lc_urban = lc_arr == 1
    arr_urban = arr * lc_urban
    
    # multiply for rural
    lc_rural = lc_arr != 1
    arr_rural = arr * lc_rural

    # meta data
    meta = rasterio.open(rst_fn).meta

    # write out urban
    meta['dtype'] = arr_urban.dtype
    out_fn = data_out+rst_nm+'_urban.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out.write_band(1, arr_urban)
    
    # write out rural 
    meta['dtype'] = arr_rural.dtype
    out_fn = data_out+rst_nm+'_rural.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out.write_band(1, arr_rural)
    print('done \n')

In [ ]:
# Make a rural/urban raster (MGUP rural value == 0)
urban_rural = modis_match.copy()
urban_rural.data = np.where(urban_rural.data == 0, 0, urban_rural.data)
urban_rural.data = np.where(urban_rural.data != 1, 0, urban_rural.data)
urban_rural.rio.to_raster(data_out+'MGUP15-rural-urban-matched.tif')

In [ ]:
# Open modis urban/rural land cover
modis_match_fn = os.path.join(data_out+'MGUP15-rural-urban-matched.tif')
modis_arr = rasterio.open(modis_match_fn).read(1)

In [ ]:
# Git matched tif files, drop MGUP
rst_fns = glob(data_out + '*_matched.tif')
rst_fns

In [ ]:
# Mask all the rasters and save them 
data_out = os.path.join('./')
for rst_fn in rst_fns:
    raster_mask(rst_fn, modis_arr, data_out)

<hr style="border-top: 0.2px solid gray; margin-top: 12pt; margin-bottom: 0pt"></hr>

# Estimate Population Exposure to Quake by Intensity 

Now we will dive into estimating how many people were impacted by the quake by earth quake intensity. 

In [ ]:
# Functions
def zone_loop(polys_in, rst_list, stats_type, col, split):
    """ Function loops through rasters, calcs zonal_stats and returns stats as a data frame.
    Args:
        polys_in = polygons
        rst_list = list of paths & fns of rasters
        stats_type = stats type for each poly gone (see zonal stats)
        col = column to merge it all
        split = where to split the file name string (e.g. _matched.tif)
    """
    
    # copy polys to write out
    polys_out = polys_in.copy()
    
    for rst in rst_list:
        
        # Get data name
        data = rst.split('/')[1].split(split)[0].split('.tif')[0]
        print('Started', data)
        
        # Run zonal stats
        zs_feats = zonal_stats(polys_in, rst, stats=stats_type, geojson_out=True)
        zgdf = gpd.GeoDataFrame.from_features(zs_feats, crs=polys_in.crs)
        
        # Rename columns and merge
        zgdf = zgdf.rename(columns={stats_type: data+'_'+stats_type})
        
        polys_out = polys_out.merge(zgdf[[col, data+'_'+stats_type]], on = col, how = 'inner')
    
    return polys_out

def poly_prep(polys_fn, col):
    "function opens earth quake polygons for zonal loop"
    
    # open
    polys = gpd.read_file(polys_fn)
    
    # subset, be sure to check the admin level
    polys = polys[['geometry', col]]
    
    return polys

## Run on Shakemap Intensity Contours (MI) from USGS for Nepal 2015

We are using USGS Shakemap intensity contours recorded during the earthquake. You can read about these data [here](https://databasin.org/datasets/9411dd80be424001ac7af150f8d3b7b0/). 

In [ ]:
# Open the shape map polygons
nepal_polys_fn = os.path.join(data_in + 'USGS_Data/Nepal/shape/mi.shp')

In [ ]:
# Subset the polygons 
col = 'PARAMVALUE'
nepal_polys = poly_prep(nepal_polys_fn, col)
nepal_polys.head()

#### Plot the MI shape file

In [ ]:
# your code here ... 

### All Data

In [ ]:
# Git tif files
rst_fns = sorted(glob(data_out + '*npl.tif'))
rst_fns

In [ ]:
# Notice we need to remove the modis data
rst_fns.remove('./modis2015_npl.tif')
rst_fns

In [ ]:
# Run zonal stats loop
nepal_polys_sum = zone_loop(nepal_polys, rst_fns, 'sum', col, '_all_NPL.tif')

In [ ]:
# Check the data
nepal_polys_sum.head()

In [ ]:
# Save the data
fn_out = os.path.join(data_out + 'nepal_quake_pop.shp')
nepal_polys_sum.to_file(fn_out)

### Urban Data

In [ ]:
# Git urban tif files
rst_fns = sorted(glob(data_out + '*urban.tif'))
rst_fns

In [ ]:
# Run zonal stats loop
nepal_polys_sum = zone_loop(nepal_polys, rst_fns, 'sum', col, '_urban_NPL.tif')

In [ ]:
# Check the data
nepal_polys_sum.head()

In [ ]:
# Save the poly sums
fn_out = os.path.join(data_out + 'nepal_urban_quake_pop.shp')
nepal_polys_sum.to_file(fn_out)

### Rural Data

In [ ]:
# Git rural tif files
rst_fns = glob(data_out + '*rural.tif')
rst_fns

In [ ]:
# Run zonal stats loop
nepal_polys_sum = zone_loop(nepal_polys, rst_fns, 'sum', col, '_rural_NPL.tif')

In [ ]:
# Check the data
nepal_polys_sum.head()

In [ ]:
#### Save the poly sums
fn_out = os.path.join(data_out + 'nepal_rural_quake_pop.shp')
nepal_polys_sum.to_file(fn_out)

## Let's double check the data
The urban and rural data should equal the total data. But let's check it out! 

In [ ]:
# All
fn_in = os.path.join(data_out + 'nepal_quake_pop.shp')
all_pop = gpd.read_file(fn_in)
all_pop.head()

In [ ]:
# Urban
fn_in = os.path.join(data_out + 'nepal_urban_quake_pop.shp')
urban_pop = gpd.read_file(fn_in)
urban_pop.head()

In [ ]:
# Rural 
fn_in = os.path.join(data_out + 'nepal_rural_quake_pop.shp')
rural_pop = gpd.read_file(fn_in)
rural_pop.head()

In [ ]:
# Check the data 
top = (rural_pop.iloc[:,1:6] + urban_pop.iloc[:,1:6]).values
bottom = all_pop.iloc[:,1:6]

In [ ]:
top / bottom

#### Question:
Explain why `top / bottom` doesn't equal one?

<hr style="border-top: 0.2px solid gray; margin-top: 12pt; margin-bottom: 0pt"></hr>

# Bar Plot
Let's look at how exposure levels, but urban and rural populations, compare across quake intensities.

**Colors** <br>
There are a ton of named colors available in [`matplotlib`](https://matplotlib.org/stable/gallery/color/named_colors.html). Check em out ane pick some!

In [ ]:
# set colors
ESRI16_c = 'blue'
GHS15_c = 'indigo'
GPWv4_c = 'deeppink'
LS15_c = 'deepskyblue'
WP16_c = 'forestgreen'

In [ ]:
# Open the shape files 
npl_all_fn = os.path.join(data_out + 'nepal_quake_pop.shp')
npl_all = gpd.read_file(npl_all_fn)

npl_rural_fn = os.path.join(data_out + 'nepal_rural_quake_pop.shp')
npl_rural = gpd.read_file(npl_rural_fn)

npl_urban_fn = os.path.join(data_out + 'nepal_urban_quake_pop.shp')
npl_urban = gpd.read_file(npl_urban_fn)

In [ ]:
# Make data
def group(df):
    
    "Group and sum population by MI ranges, args is df quake pop"
    
    iv = df[(df['PARAMVALUE'] >= 4) & (df['PARAMVALUE'] < 5)].iloc[:,1:6].sum(axis = 0)
    v = df[(df['PARAMVALUE'] >= 5) & (df['PARAMVALUE'] < 6)].iloc[:,1:6].sum(axis = 0)
    vi = df[(df['PARAMVALUE'] >= 6) & (df['PARAMVALUE'] < 7)].iloc[:,1:6].sum(axis = 0)
    vii = df[df['PARAMVALUE'] >= 7].iloc[:,1:6].sum(axis = 0)
    
    out = pd.DataFrame()
    out['iv'] = iv
    out['v'] = v
    out['vi'] = vi
    out['vii'] = vii
    
    out = out.transpose()
    return out

In [ ]:
group(npl_rural)

In [ ]:
# Make bar plot 
fig, axs = plt.subplots(1, 1, figsize = (12, 8), sharex=True)
ws = 0.25
fig.subplots_adjust(wspace=ws)
scale = 10**6

# All Quake
data = group(npl_all)

# Bar locations
a = [1-.3,2-.3,3-.3, 4-.3]
b = [1-.15,2-.15,3-.15,4-.15]
c = [1,2,3,4]
d = [1+.15,2+.15,3+.15,4+.15]
e = [1+.3,2+.3,3+.3,4+.3]

# plots
plt.bar(a, data.esri2015_n / scale, width=0.12, align='center', alpha  = 0.5, color = ESRI16_c, ec = 'black')
plt.bar(b, data.ghs2015_np / scale, width=0.12, align='center', alpha  = 0.6, color = GHS15_c, ec = 'black')
plt.bar(c, data.gpw2015_np / scale, width=0.12, align='center', alpha  = 0.7, color = GPWv4_c, ec = 'black')
plt.bar(d, data.ls2015_npl / scale, width=0.12, align='center', alpha  = 0.8, color = LS15_c, ec = 'black')
plt.bar(e, data.wp2016_npl / scale, width=0.12, align='center', alpha  = 0.9, color = WP16_c, ec = 'black')

# Fake plot for rural hatch legend 
plt.bar(e, data.wp2016_npl / scale, width=0.12, align='center', alpha  = 0, color = 'white', ec = 'black',hatch = "///")

# rural quake
data = group(npl_rural)
plt.bar(a, data.ESRI16_mat / scale, width=0.12, align='center', alpha  = 0.5, color = ESRI16_c, ec = 'black', hatch = "///")
plt.bar(b, data.GHS15_matc / scale, width=0.12, align='center', alpha  = 0.6, color = GHS15_c, ec = 'black', hatch = "///")
plt.bar(c, data.GPWv4_matc / scale, width=0.12, align='center', alpha  = 0.7, color = GPWv4_c, ec = 'black', hatch = "///")
plt.bar(d, data.LS15_match / scale, width=0.12, align='center', alpha  = 0.8, color = LS15_c, ec = 'black', hatch = "///")
plt.bar(e, data.WP16_match / scale, width=0.12, align='center', alpha  = 0.9, color = WP16_c, ec = 'black', hatch = "///")

# legend
legend_elements = [Patch(facecolor=ESRI16_c, alpha = 0.5, edgecolor=None, label='WPE-15'),
                  Patch(facecolor=GHS15_c, alpha = 0.6, edgecolor=None, label='GHSL-15'),
                  Patch(facecolor=GPWv4_c, alpha = 0.7, edgecolor=None, label='GPW-15'),
                  Patch(facecolor=LS15_c, alpha = 0.8, edgecolor=None, label='LS-15'),
                  Patch(facecolor= WP16_c, alpha = 0.9, edgecolor=None, label='WP-16'),
                  Patch(facecolor= 'white', alpha = 0.9,  hatch = '///', edgecolor='black', label='rural pop')]
plt.legend(handles = legend_elements, bbox_to_anchor=(1, 1.02), loc='upper left', ncol=1, fontsize = 15);

# Labels / Titles
axs.set_title('Nepal 2015 Earthquake Impact', size = 20)
axs.set_xlabel('Instrumental Intesnity', fontsize = 15)
axs.set_ylabel('Total Population [millions]', fontsize = 15)

# Ticks
ticks_bar = ['>=4', ' >=5', '>=6', '>=7'];
plt.xticks([1,2,3,4], ticks_bar, fontsize = 15);
plt.yticks(fontsize = 15);

In [ ]:
# save it out
fig_out = os.path.join(data_out + 'Figure.png')
plt.savefig(fig_out, dpi = 300, facecolor = 'white', bbox_inches='tight')